<a href="https://colab.research.google.com/github/ch23s020/assignment2/blob/main/Assignment2_partb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb


In [ ]:
import wandb
import torch
from tqdm import tqdm
from torch import nn, optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_img_shape(model_name):

    if model_name == 'resnet50':

        return 224, 224

    elif model_name == 'inception_v3':

        return 299, 299

    elif model_name == 'inception_resnet_v2':

        return 299, 299

    elif model_name == 'xception':

        return 299, 299

    else:
        raise ValueError("Invalid model name")

def train():
    # hyperparameters

    project_name = 'cs6910_assignment2_cnn_partb'

    model_name = 'resnet50'

    epochs = 5

    batch_size = 16

    optimizer_name = 'Adam'

    data_augmentation = False

    pretrained = True

    learning_rate = 3e-4

    # Initialize wandb run

    wandb.init(project=project_name)

    # Data transform according to model

    data_transforms = transforms.Compose([

        transforms.Resize(load_img_shape(model_name)),

        transforms.ToTensor(),
    ])

    # Loading data
    train_data = ImageFolder(root='/content/drive/MyDrive/nature_DL/inaturalist_12K/train', transform=data_transforms)

    test_data = ImageFolder(root='/content/drive/MyDrive/nature_DL/inaturalist_12K/val', transform=data_transforms)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Loading model
    model = getattr(models, model_name)(pretrained=pretrained)

    num_features = model.fc.in_features

    model.fc = nn.Linear(num_features, len(train_data.classes))

    model = model.to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):

        model.train()

        train_correct, train_loss = .0, .0

        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):

            # Get data to device
            data, targets = data.to(device), targets.to(device)

            # Forward_prop
            outputs = model(data)

            loss = criterion(outputs, targets)

            # Backward_prop
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            # Accuracy Calculation
            _, predictions = torch.max(outputs, 1)

            train_correct += (predictions == targets).sum().item()

            train_loss += loss.item()

            # Log metrics to wandb

            wandb.log({
                'train_loss': loss.item(),

                'train_acc': (predictions == targets).float().mean().item(),
            })

        train_loss /= len(train_loader)

        train_acc = train_correct / len(train_data)

        # Evaluate on test data
        model.eval()
        test_correct = 0
        with torch.no_grad():
            for data, targets in test_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == targets).sum().item()

        test_acc = test_correct / len(test_data)

        # Log epoch metrics to wandb

        wandb.log({

            'epoch': epoch,

            'train_acc': train_acc,

            'train_loss': train_loss,

            'test_acc': test_acc,
        })

        print(f'\nEpoch {epoch}, train_acc {train_acc}, train_loss {train_loss}, test_acc {test_acc}')

    wandb.finish()

# Call the train function

train()
